Part of this code utilizes the `segment-geospatial` package, which is a Python package designed for segmenting geospatial data using the Segment Anything Model (SAM). This package was created by Professor Dr. Qiusheng Wu.

The `segment-geospatial` package has been adapted from the `segment-anything-eo` repository, originally authored by Aliaksandr Hancharenka. The main goal of the `segment-geospatial` package is to simplify the process of applying SAM to geospatial data analysis, making it more accessible and requiring minimal coding effort.

To use this package, it's available for download and installation via PyPI and conda-forge.

For more information, details, and examples on how to use this package, you can visit Professor Wu's GitHub page at [https://github.com/opengeos/segment-geospatial](https://github.com/opengeos/segment-geospatial).

<a href="https://colab.research.google.com/github/LucasOsco/AI-RemoteSensing/blob/main/SegmentGeospatial_Point_vShared.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

***Connect to your Google Drive***

In [ ]:
from google.colab import drive 
drive.mount('/content/gdrive')

***Install both dependencies***

In [ ]:
pip install segment-geospatial leafmap localtileserver

In [ ]:
pip install pycrs

***Import libraries***

In [ ]:
import os
import zipfile
import leafmap
import geopandas as gpd
import rasterio
import glob
from rasterio.merge import merge
from samgeo import SamGeo, tms_to_geotiff
from segment_anything import sam_model_registry

***Load image and shapefile from Drive***

In [ ]:
# Read the image
image = 'gdrive/My Drive/Colab Notebooks/Image/roi_image.tif' # Switch to your directory instead

# Read the shapefile
shapefile = 'gdrive/My Drive/Colab Notebooks/Shape/roi_point.shp' # Switch to your directory instead
gdf = gpd.read_file(shapefile)

# Convert the GeoDataFrame to a list of [x, y] pairs
point_coords = [[point.x, point.y] for point in gdf.geometry]

***Specify the file path to the model checkpoint***

In [ ]:
# Load the model from Drive
out_dir = os.path.join('gdrive/My Drive/Colab Notebooks/Model/') # Switch to your directory instead
checkpoint = os.path.join(out_dir, 'sam_vit_h_4b8939.pth')

***Initialize SAM class***

In [ ]:
#Set automatic=False to disable the SamAutomaticMaskGenerator and enable the SamPredictor
sam = SamGeo(
    model_type="vit_h",
    checkpoint=checkpoint,
    automatic=False,
    sam_kwargs=None,
)

#Specify the image to segment
sam.set_image(image)

***Loop SAM through each point feature and store the individual masks into a .zip file and save it on Drive***

In [ ]:
# Prepare zip file to be saved
zip_name = 'masks.zip'
zipf = zipfile.ZipFile(zip_name, 'w', zipfile.ZIP_DEFLATED)

# Loop through each point in point_coords
for i, point in enumerate(point_coords):
    # Predict and save to a unique file per point
    output_file = f'mask_{i}.tif'
    sam.predict([point], point_labels=1, point_crs="EPSG:4326", output=output_file)
    
    # Add file to zip
    zipf.write(output_file)

# Close the zip file after all files are added
zipf.close()

***Combine all the masks into a single mosaic and save it on Drive***

In [ ]:
# Get a list of your mask files
search_criteria = "mask_*.tif"
q = os.path.join(search_criteria)
dem_fps = glob.glob(q)

# Read them into a list of rasterio dataset objects
src_files_to_mosaic = []
for fp in dem_fps:
    src = rasterio.open(fp)
    src_files_to_mosaic.append(src)

# Merge them into a single file
mosaic, out_trans = merge(src_files_to_mosaic)

# Copy the metadata
out_meta = src.meta.copy()

# Update the metadata
out_meta.update({"driver": "GTiff",
                 "height": mosaic.shape[1],
                 "width": mosaic.shape[2],
                 "transform": out_trans,
                 "crs": "EPSG:4326"
                 }
                )

# Define the path to the Google Drive directory where you want to save the file.
output_dir = '/content/gdrive/My Drive/Colab Notebooks/Output' # Switch to your directory instead

# Make sure to join the output directory with the output filename.
output_file = os.path.join(output_dir, 'mosaic.tif')

# Write the mosaic raster to disk in your Google Drive.
with rasterio.open(output_file, "w", **out_meta) as dest:
    dest.write(mosaic)

# Close the files
for src in src_files_to_mosaic:
    src.close()

***Visualize the image segmentation with input points***

In [ ]:
# Display the results
mosaic = 'gdrive/My Drive/Colab Notebooks/Output/mosaic.tif' # Switch to your directory instead
style={'color': '#a37aa9',}

m = leafmap.Map(center=[-22.17615, -51.253043], zoom=14)
m.add_raster(mosaic, layer_name="Mask Mosaic", nodata=0, cmap="Greens", opacity=0.7)
m.add_vector(shapefile, layer_name='Vector', style=style)
m